# **IMPORTANDO LIBRERIAS**

In [1]:
import sys
import os
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os
from sqlalchemy.orm import declarative_base, relationship
from sqlalchemy import create_engine, Column, Integer, String, VARCHAR,Boolean, TIMESTAMP,Float
from sqlalchemy.types import DateTime
from sqlalchemy.orm import sessionmaker


sys.path.append(os.path.abspath(os.path.join('..')))

In [2]:
load_dotenv()  # Carga las variables del archivo .env

config = {
    'host': os.getenv('HOST'),
    'user': os.getenv('USER'),
    'password': os.getenv('PASSWORD'),
    'database': os.getenv('DATABASE'),
    'port': os.getenv('PORT')
}

In [3]:
# Crear la URL de conexión para SQLAlchemy
db_url = f"mysql+pymysql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/caso_8"

# Crear el motor de SQLAlchemy
engine = create_engine(db_url, echo=True)



In [4]:
Base = declarative_base()


# **1. GENERACIÓN DE TABLAS**

In [5]:
class InterestMap(Base):

    __tablename__ = 'interest_map'
    __table_args__= {'extend_existing':True}

    id = Column(Integer, primary_key=True)
    interest_name = Column(VARCHAR(255))
    interest_summary = Column(VARCHAR(255))
    created_at = Column(TIMESTAMP)
    last_modified = Column(TIMESTAMP)

class InterestMetrics(Base):
    __tablename__ = 'interest_metrics'
    __table_args__ = {'extend_existing': True}

    id = Column(Integer, primary_key=True, autoincrement=True)
    _month = Column(VARCHAR(4))
    _year = Column(VARCHAR(4))
    month_year = Column(VARCHAR(7))
    interest_id = Column(VARCHAR(5))
    composition = Column(Float)
    index_value = Column(Float)
    ranking = Column(Integer)
    percentile_ranking = Column(Float)

Base.metadata.create_all(engine)


2025-05-05 14:11:10,756 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-05-05 14:11:10,757 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-05 14:11:10,758 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-05-05 14:11:10,759 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-05 14:11:10,760 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-05-05 14:11:10,760 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-05 14:11:10,762 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-05 14:11:10,762 INFO sqlalchemy.engine.Engine DESCRIBE `caso_8`.`interest_map`
2025-05-05 14:11:10,762 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-05 14:11:10,765 INFO sqlalchemy.engine.Engine DESCRIBE `caso_8`.`interest_metrics`
2025-05-05 14:11:10,765 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-05 14:11:10,767 INFO sqlalchemy.engine.Engine 
CREATE TABLE interest_map (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	interest_name VARCHAR(255), 
	interest_summary VARCHAR(255), 
	c

## **1.1 CARGANDO LOS DATOS**

In [6]:
from utils import funciones_utiles

interest_map_data = funciones_utiles.load_text_as_tuples('data/interest_map_data.txt')
interest_metrics_data = funciones_utiles.load_text_as_tuples('data/interest_metrics_data.txt')

In [7]:
# Creando la session
Session = sessionmaker(bind=engine)
session = Session()

### **1.1.1 Cargando datos `interest_map`**

In [8]:
# Obtener los ids existentes de InterestMap
existing_ids = {row.id for row in session.query(InterestMap.id).all()}

objects = [
    InterestMap(
        id=id,
        interest_name=interest_name,
        interest_summary=interest_summary,
        created_at=created_at,
        last_modified=last_modified
    )
    for id, interest_name, interest_summary, created_at, last_modified in interest_map_data
    if id not in existing_ids
]

if objects:
    session.bulk_save_objects(objects)
    session.commit()
else:
    print("No hay nuevos registros para insertar en InterestMap.")

2025-05-05 14:11:11,185 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-05 14:11:11,189 INFO sqlalchemy.engine.Engine SELECT interest_map.id AS interest_map_id 
FROM interest_map
2025-05-05 14:11:11,191 INFO sqlalchemy.engine.Engine [generated in 0.00113s] {}
2025-05-05 14:11:11,220 INFO sqlalchemy.engine.Engine INSERT INTO interest_map (id, interest_name, interest_summary, created_at, last_modified) VALUES (%(id)s, %(interest_name)s, %(interest_summary)s, %(created_at)s, %(last_modified)s)
2025-05-05 14:11:11,220 INFO sqlalchemy.engine.Engine [generated in 0.00467s] [{'id': '1', 'interest_name': 'Fitness Enthusiasts', 'interest_summary': 'Consumers using fitness tracking apps and websites.', 'created_at': '2016-05-26 14:57:59', 'last_modified': '2018-05-23 11:30:12'}, {'id': '2', 'interest_name': 'Gamers', 'interest_summary': 'Consumers researching game reviews and cheat codes.', 'created_at': '2016-05-26 14:57:59', 'last_modified': '2018-05-23 11:30:12'}, {'id': '3', 'interest

### **1.1.1 Cargando datos `interest_metrics`**

In [9]:
import numpy as np
def clean_value(value, dtype='str'):
    if value in ('NULL', '', np.nan):
        return None
    if dtype == 'int':
        return int(value)
    if dtype == 'float':
        return float(value)
    return value  # default: string

In [ ]:
# Obtener los ids existentes de InterestMap
existing_ids = {row.id for row in session.query(InterestMetrics.id).all()}

objects = [
    InterestMetrics(
        _month = _month if _month != 'NULL' else np.nan,
        _year =_year if _year != 'NULL' else np.nan,
        month_year = month_year if month_year != 'NULL' else np.nan,
        interest_id = interest_id,
        composition = composition,
        index_value = index_value,
        ranking = ranking,
        percentile_ranking = percentile_ranking
    )
    for _month, _year, month_year, interest_id, composition, index_value, ranking, percentile_ranking  in interest_metrics_data
    if id not in existing_ids
]

if objects:
    session.bulk_save_objects(objects)
    session.commit()
else:
    print("No hay nuevos registros para insertar en InterestMap.")

2025-05-05 14:11:11,310 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-05 14:11:11,312 INFO sqlalchemy.engine.Engine SELECT interest_metrics.id AS interest_metrics_id 
FROM interest_metrics
2025-05-05 14:11:11,313 INFO sqlalchemy.engine.Engine [generated in 0.00108s] {}
2025-05-05 14:11:11,779 INFO sqlalchemy.engine.Engine INSERT INTO interest_metrics (_month, _year, month_year, interest_id, composition, index_value, ranking, percentile_ranking) VALUES (%(_month)s, %(_year)s, %(month_year)s, %(interest_id)s, %(composition)s, %(index_value)s, %(ranking)s, %(percentile_ranking)s)
2025-05-05 14:11:11,779 INFO sqlalchemy.engine.Engine [generated in 0.05026s] [{'_month': '7', '_year': '2018', 'month_year': '07-2018', 'interest_id': '32486', 'composition': '11.89', 'index_value': '6.19', 'ranking': '1', 'percentile_ranking': '99.86'}, {'_month': '7', '_year': '2018', 'month_year': '07-2018', 'interest_id': '6106', 'composition': '9.93', 'index_value': '5.31', 'ranking': '2', 'percent

# Revision datos

In [11]:
query = '''

   SELECT 
    SUM(CASE WHEN id = 'NULL' OR id IS NULL OR id = '' THEN 1 ELSE 0 END) AS id_nulls,
    SUM(CASE WHEN interest_name = 'NULL' OR interest_name IS NULL OR interest_name = '' THEN 1 ELSE 0 END) AS interest_name_nulls,
    SUM(CASE WHEN interest_summary = 'NULL' OR interest_summary IS NULL OR interest_summary = '' THEN 1 ELSE 0 END) AS interest_summary_nulls,
    SUM(CASE WHEN created_at IS NULL THEN 1 ELSE 0 END) AS created_at_nulls,
    SUM(CASE WHEN last_modified IS NULL THEN 1 ELSE 0 END) AS last_modified_nulls
FROM interest_map;

'''

pd.read_sql_query(query, con=engine)

2025-05-05 14:11:12,280 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-05 14:11:12,280 INFO sqlalchemy.engine.Engine 

   SELECT 
    SUM(CASE WHEN id = 'NULL' OR id IS NULL OR id = '' THEN 1 ELSE 0 END) AS id_nulls,
    SUM(CASE WHEN interest_name = 'NULL' OR interest_name IS NULL OR interest_name = '' THEN 1 ELSE 0 END) AS interest_name_nulls,
    SUM(CASE WHEN interest_summary = 'NULL' OR interest_summary IS NULL OR interest_summary = '' THEN 1 ELSE 0 END) AS interest_summary_nulls,
    SUM(CASE WHEN created_at IS NULL THEN 1 ELSE 0 END) AS created_at_nulls,
    SUM(CASE WHEN last_modified IS NULL THEN 1 ELSE 0 END) AS last_modified_nulls
FROM interest_map;


2025-05-05 14:11:12,281 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-05 14:11:12,285 INFO sqlalchemy.engine.Engine ROLLBACK


,id_nulls,interest_name_nulls,interest_summary_nulls,created_at_nulls,last_modified_nulls
0,0.0,0.0,20.0,0.0,0.0


In [ ]:
query = '''

    SELECT 
        SUM(CASE WHEN _month = 'NULL' OR _month IS NULL OR _month = '' THEN 1 ELSE 0 END) AS month_nulls,
        SUM(CASE WHEN _year = 'NULL' OR _year IS NULL OR _year = '' THEN 1 ELSE 0 END) AS _year_nulls,
        SUM(CASE WHEN month_year = 'NULL' OR month_year IS NULL OR month_year = '' THEN 1 ELSE 0 END) AS month_year_nulls,
        SUM(CASE WHEN interest_id = 'NULL' OR interest_id IS NULL OR interest_id = '' THEN 1 ELSE 0 END) AS interest_id_nulls,
        SUM(CASE WHEN index_value IS NULL THEN 1 ELSE 0 END) AS index_value_nulls,
        SUM(CASE WHEN ranking IS NULL THEN 1 ELSE 0 END) AS ranking_nulls,
        SUM(CASE WHEN percentile_ranking IS NULL THEN 1 ELSE 0 END) AS percentile_ranking_nulls

    FROM interest_metrics;

'''
pd.read_sql_query(query, con=engine)


2025-05-05 14:15:08,853 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-05 14:15:08,854 INFO sqlalchemy.engine.Engine 

SELECT 
    SUM(_month = 'NULL') AS month_nulls,
    SUM(_year = 'NULL') AS _year_nulls,
    SUM(month_year = 'NULL') AS month_year_nulls,
    SUM(interest_id = 'NULL') AS interest_id_nulls,
    SUM(index_value = 'NULL') AS index_value_nulls,
    SUM(ranking = 'NULL') AS ranking_nulls,
    SUM(percentile_ranking = 'NULL') AS percentile_ranking_nulls
FROM interest_metrics;
;


2025-05-05 14:15:08,854 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-05 14:15:08,873 INFO sqlalchemy.engine.Engine ROLLBACK


,month_nulls,_year_nulls,month_year_nulls,interest_id_nulls,index_value_nulls,ranking_nulls,percentile_ranking_nulls
0,0.0,0.0,0.0,0.0,0.0,0.0,14.0


In [24]:
query = '''

SELECT COUNT(*) FROM interest_metrics WHERE _month IS NULL;
'''
pd.read_sql_query(query, con=engine)


2025-05-05 14:16:34,009 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-05 14:16:34,012 INFO sqlalchemy.engine.Engine 

SELECT COUNT(*) FROM interest_metrics WHERE _month IS NULL;

2025-05-05 14:16:34,012 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-05 14:16:34,018 INFO sqlalchemy.engine.Engine ROLLBACK


,COUNT(*)
0,0
